# Data Wrangling


This notebook will be used to acquire and perform wrnagling exploration for the data to be used in this study. I test using the Yelp API but 

In [ ]:
import requests
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('./YelpAPIKey.env')

load_dotenv(dotenv_path=dotenv_path)

API_CLIENT_ID = os.getenv('Yelp_Client_ID')
API_CLIENT_KEY = os.getenv('Yelp_API_Key')

In [ ]:
# Example business ID
business_ids = '0qNpTGTcqPwOLi2hADx4Xw'

ENDPOINT_BUSINESS = 'https://api.yelp.com/v3/businesses/search'
ENDPOINT_REVIEWS = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(business_ids)

HEADERS = {'Authorization': 'bearer %s' % API_CLIENT_KEY}

In [ ]:
response = requests.get(url=ENDPOINT_REVIEWS, headers=HEADERS)

In [ ]:
data_test = response.json()
print(data_test)

#### After trying the API, there are a few issues that I do not want to deal with: 
- The text is limited to 120 characters which will significantly worsen my sentiment analysis
- The API only allows 5,000 calls per 24 hours which owuld make data collection take a long time
- Yelp offers their open dataset with vastly more business and reviews data for free for educational purposes

I will import that data and use it for the study instead. 


In [2]:
import patoolib
import tarfile
import os
import json
import csv
import pandas as pd

path = r'C:\Users\ziggy\Springboard\Python_Projects\Yelp Restaurant Analysis\Data'

In [111]:
os.chdir(path)

t1 = os.listdir('.')[1]
t1

'Dataset_User_Agreement.pdf'

In [112]:
tar = tarfile.open(t1)
tar.extractall()
tar.close()

ReadError: file could not be opened successfully

In [5]:
business_json_path = r'C:\Users\ziggy\Springboard\Python_Projects\Yelp Restaurant Analysis\Data\yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)


ValueError: Expected object or value

In [4]:
df_b.head()

NameError: name 'df_b' is not defined

##### The first step will be to get the infomration about restaurants.
I will need to use the categories column to filter all the data to: "Food", "restaurants", etc. 

I will then be able to create a dataframe that is specific to restaurants with business Ids that I can match to the reviews for the sentiment analysis. 

I will then be able to see if there is any correlation between attributes and ratings, geosptial analysis for the types of businesses. etc.

In [26]:
df_b.categories

0         Doctors, Traditional Chinese Medicine, Naturop...
1         Shipping Centers, Local Services, Notaries, Ma...
2         Department Stores, Shopping, Fashion, Home & G...
3         Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4                                 Brewpubs, Breweries, Food
                                ...                        
150341                           Nail Salons, Beauty & Spas
150342    Pets, Nurseries & Gardening, Pet Stores, Hobby...
150343    Shopping, Jewelry, Piercing, Toy Stores, Beaut...
150344    Fitness/Exercise Equipment, Eyewear & Optician...
150345    Beauty & Spas, Permanent Makeup, Piercing, Tattoo
Name: categories, Length: 150346, dtype: object

In [27]:
pattern = 'food|restaurant'
df_food = df_b['categories'].str.contains(pattern, case=False, na=False)

In [28]:
df_r = df_b[df_food]

In [29]:
df_r.describe()

,latitude,longitude,stars,review_count,is_open
count,64629.000000,64629.000000,64629.000000,64629.000000,64629.000000
mean,36.969005,-88.096469,3.545962,76.600891,0.690000
std,6.019033,14.025379,0.853905,173.369068,0.462497
min,27.564457,-120.083748,1.000000,5.000000,0.000000
25%,32.207352,-90.247514,3.000000,12.000000,0.000000
50%,39.479360,-86.048740,3.500000,28.000000,1.000000
75%,39.958126,-75.346993,4.000000,75.000000,1.000000
max,53.679197,-74.661348,5.000000,7568.000000,1.000000


In [1]:
categories = pd.DataFrame(df_r[['categories', 'stars']])

NameError: name 'pd' is not defined

##### Initial thoughts
The average rating is 3.54. 



In [30]:
df_r.isna().sum()

business_id        0
name               0
address            0
city               0
state              0
postal_code        0
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
attributes       877
categories         0
hours           9336
dtype: int64

###### Inerestingly, within our data there are several establishments missing information for attributes and hours. I am going to drop them in the interest of moving on with the analysis. I was hoping there might be some was to use the missing store hours to show restaruatns that had closed down but that is what the "is_open" column is for. 

In [31]:
df_r[df_r['hours'].isnull()]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
24,4iRzR7OaS-QaSXuvYxEGKA,Super Dog,1160 Gallatin Pike S,Nashville,TN,37115,36.248570,-86.719986,4.0,6,0,"{'RestaurantsReservations': 'False', 'Restaura...","Hot Dogs, Restaurants",None
36,2xVsWBNFwZOxIOdd9Mwnww,Cheeseburger In Paradise,116 N Pottstown Pike,Exton,PA,19341,40.029962,-75.630607,2.5,20,0,"{'NoiseLevel': 'u'average'', 'HasTV': 'True', ...","Restaurants, Burgers",None
54,0qNpTGTcqPwOLi2hADx4Xw,Charlie's Market,2815 E Sligh Ave,Tampa,FL,33610,28.010360,-82.430042,3.0,9,1,"{'BusinessParking': '{'garage': False, 'street...","Food, Grocery, Convenience Stores",None
62,L_TT0BFmFwORAMaA83K54A,Village Tap Room,838 Broad Ripple Ave,Indianapolis,IN,46220,39.869911,-86.143577,2.5,23,0,"{'Alcohol': 'u'none'', 'BestNights': '{'monday...","Gastropubs, Cocktail Bars, Beer Bar, Bars, Res...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150261,Y0TWCjiiXlFi2XO2cFrUzQ,Connexxions Cafe,12644 137 Avenue NW,Edmonton,AB,T5L 4Y5,53.600573,-113.540273,4.5,13,0,"{'RestaurantsReservations': 'True', 'Alcohol':...","Hobby Shops, Tabletop Games, Coffee & Tea, Foo...",None
150262,-bZQH8yjm7ntTyGeLQwh8Q,Farmer's Kitchen Restaurant,3500 E Bay Dr,Largo,FL,33771,27.916787,-82.750395,4.0,6,0,"{'RestaurantsReservations': 'True', 'Restauran...","Sandwiches, Restaurants, Diners",None
150272,8BUr8GviR2o_b-brO21wwQ,Kyle's Isla Vista,900 Embarcadero Del Mar,Isla Vista,CA,93117,34.412966,-119.856886,2.5,12,1,"{'OutdoorSeating': 'True', 'BusinessAcceptsBit...","Restaurants, Burgers, Food, Ice Cream & Frozen...",None
150273,awRAcg8-5OqlULhDiIEFfA,Rustica Restaurant Pizzera,5860 York Rd,New Hope,PA,18938,40.346275,-75.035753,3.0,21,0,"{'Alcohol': 'u'none'', 'HasTV': 'True', 'Outdo...","Pizza, Restaurants",None


In [32]:
df_res = df_r.dropna()

In [33]:
df_res.to_csv('Restaurants')

In [66]:
df_business_ids = df_res.business_id

In [64]:
df_buisness_ids.head()

3     MTSW4McQd7CbVtyjqoe9mw
4     mWMc6_wTdE0EUBKIGXDVfA
5     CF33F8-E6oudUQ46HnavjQ
9     bBDDEgkFA1Otx9Lfe7BZUQ
11    eEOYSgkmpB90uNA7lDOMRA
Name: business_id, dtype: object

In [70]:
reviews_json_path = r'C:\Users\ziggy\Springboard\Python_Projects\Yelp Restaurant Analysis\Data\yelp_academic_dataset_review.json'
reviews = pd.read_json(reviews_json_path, lines=True, chunksize=1000000)


In [71]:
for chunk in reviews:
    print(chunk)

                     review_id                 user_id  \
0       KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA   
1       BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q   
2       saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A   
3       AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ   
4       Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ   
...                        ...                     ...   
999995  t-2o35kr7Q9DSaeuKhaDuQ  oX7o1TH0PHUWp9r9ry9_vw   
999996  fLIwWCvdul9PNWYfJt5QWA  v8wlapFKVLs2qTYCGhCdiw   
999997  ETAiy6wEM-r9ve4SKDhBpg  rLlYc1RzIBnOmnX3AbpEYw   
999998  8OgvSXuc6KjAt2fSz9LuzA  eEH-8CEPU5ndPxDGzVfHiQ   
999999  TQoVzNXqDkiDXgzlw8cyaQ  HYmGwYXvcYmW7dDjuWKJfw   

                   business_id  stars  useful  funny  cool  \
0       XQfwVwDr-v0ZS3_CbbE5Xw      3       0      0     0   
1       7ATYjTIgM3jUlt4UM3IypQ      5       1      0     1   
2       YjUWPpI6HXG530lwP-fb2A      3       0      0     0   
3       kxX2SOes4o-D3ZQBkiMRfA      5       1      0   

                      review_id                 user_id  \
4000000  tNx450494cTULI7CmykEyA  iOrUgvydcxIDlW78bchkRA   
4000001  LLus2Mkw4C3PgdPIbm2WoQ  9g2yOSgIpHq3gcpX756HeQ   
4000002  LQelK02_SN2nkvuhbLRpkg  BsMKUneO8WI0NIuXTnLVaQ   
4000003  lajxWuowVIOSTRYeSoeFdQ  u5UOl-Jsq1ozwWX84m_kBQ   
4000004  W-1q9mQNlfXTDtd9DPoqgA  Xv6bH6p63DUheLu5b5ngfg   
...                         ...                     ...   
4999995  ylWAkYNyMUE89wZi3mfF_w  SCqdOpu99eB8si3O4VNGZw   
4999996  RFKsfcOutfhFM9V_K8FUUw  T9nDMe9mljDsAV7MvxcvOw   
4999997  uuf4-lsOdDNGaxGX7kdKvQ  jh7CWRWIHNPR_n9GQn6Qww   
4999998  lHXmsAIvC8hfA0fl3qKlOQ  NIhcRW6DWvk1JQhDhXwgOQ   
4999999  VOTAzkPCl69iZCKg7VPQog  ePR435P_LXoEglMnvx5Wtw   

                    business_id  stars  useful  funny  cool  \
4000000  LU7WfvB9o5_JSW5IMvGedg      5       0      0     0   
4000001  6_T2xzR74JqGCTPefAD8Tw      3       1      0     0   
4000002  h7TO_IsmLCYmKKDVOOIeFw      5       0      0     0   
4000003  ZfAILO2iad90fCDtratcKg      4 

In [73]:
reviews = pd.read_json(reviews_json_path, lines=True, chunksize=1000000)
df_rev = pd.concat(reviews, ignore_index=True)
df_rev.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [84]:
m = df_rev.business_id.isin(df_res.business_id)

df_rev = df_rev[m]

In [87]:
df_rev = df_rev[['business_id', 'stars', 'text', 'date']]

In [88]:
df_rev.to_csv('Reviews')